In [14]:
import os
from pathlib import Path
import shutil

In [15]:
%pwd

'c:\\Users\\jatin\\Desktop\\internship project\\Alzimer_disease_detection'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\jatin\\Desktop\\internship project\\Alzimer_disease_detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    data_path: Path
    
    



In [6]:
from src.Alzimer_disease_classifier.constants import *
from src.Alzimer_disease_classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifact_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            data_path=config.data_path, 
        )

        return data_ingestion_config
        




In [8]:
import os 
from src.Alzimer_disease_classifier.logger import logging
from src.Alzimer_disease_classifier.utils.common import get_size
import sys

In [9]:
from src.Alzimer_disease_classifier.exception import RenelException

In [45]:
import os
import sys
import shutil
from pathlib import Path
from src.Alzimer_disease_classifier.logger import logging
from src.Alzimer_disease_classifier.exception import RenelException

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def get_data_from_folder(self, data_path: Path):
        """
        Copies the entire directory structure (including subfolders like `mild` and `none`)
        from `data_path` to `config.root_dir`, preserving folder hierarchy.
        """
        logging.info(f"Copying data from {data_path} to {self.config.root_dir}")
        dest_dir = Path(self.config.root_dir)

        try:
            # Ensure source directory exists
            if not data_path.exists():
                raise RenelException(f"Source path {data_path} does not exist.", sys)
            
            # Create root destination directory
            dest_dir.mkdir(parents=True, exist_ok=True)

            # Walk through the source directory
            for root, dirs, files in os.walk(data_path):
                # Get relative path from source to current directory
                relative_path = os.path.relpath(root, data_path)
                logging.info(f"Relative path: {relative_path}")
                dest_subdir = dest_dir / relative_path
                logging.info(f"Destination subdirectory: {dest_subdir}")

                # Create corresponding subdirectory in destination
                dest_subdir.mkdir(parents=True, exist_ok=True)
                logging.info(f"Created directory: {dest_subdir}")

                # Copy files to the destination subdirectory
                for file in files:
                    src_file = Path(root) / file
                    dest_file = dest_subdir / file

                    if not dest_file.exists():
                        shutil.copy(src_file, dest_subdir)
                        # logging.info(f"Copied: {src_file} → {dest_file}")
                    else:
                        logging.warning(f"Skipped (already exists): {dest_file}")

            logging.info(f"Data copied successfully to {dest_dir}")
            return str(dest_dir)
        
        except Exception as e:
            logging.error(f"Error: {e}")
            raise RenelException(e, sys)

In [44]:
for root, dirs, files in os.walk('alzimer_img'):
    rel = os.path.relpath(root,'alzimer_img')
    print(rel)

.
mild
none


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [19]:
from src.Alzimer_disease_classifier.exception import RenelException

In [46]:
from pathlib import Path

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_path = Path(data_ingestion_config.data_path)
    data_ingestion.get_data_from_folder(data_path=data_path)

except FileNotFoundError as e:
    print(f"File not found: {e.filename}")
    raise RenelException(e, sys)
except Exception as e:
    raise RenelException(e, sys)